# Polynomial chaos sensitivity to the degree

## Introduction

In this example, we observe the sensitivity of the polynomial chaos expansion to the total degree of the polynomial.
More precisely, we observe how this impacts the $Q^2$ predictivity coefficient.

We consider the example of the cantilever beam. We create a sparse polynomial chaos with a linear enumeration rule and the family of orthogonal polynomials corresponding to each input variable.



In [1]:
import openturns as ot
import numpy as np
import openturns.viewer
import pylab as pl
pl.rcParams['figure.dpi']= 500

The following parameter value leads to fast simulations.



In [2]:
maxDegree = 6

In [3]:
distribution_alpha = ot.Normal(101.6612, 4.0)
distribution_alpha.setDescription([r"$\alpha$"])
#
distribution_beta = ot.Normal(0.0233, 0.0005)
distribution_beta.setDescription([r"$\beta$"])
#
distribution_epsilon_e = ot.Triangular(0.2, 0.3, 0.5)
distribution_epsilon_e.setDescription([r"$\epsilon_e$"])
#
distribution_epsilon_c = ot.Triangular(0.0, 0.05, 0.3)
distribution_epsilon_c.setDescription([r"$\epsilon_c$"])
#
dp_factor = 1.0e-6
distribution_dp = ot.Triangular(0.5 * dp_factor, 5.0 * dp_factor, 10.0 * dp_factor)
distribution_dp.setDescription([r"$d_p$"])
#
distribution_gamma_p0 = ot.Triangular(1.0e-09, 4.5e-09, 8.0e-09)
distribution_gamma_p0.setDescription([r"$\Gamma_p(0)$"])
#
distribution_av = ot.Triangular(0.1e-4, 7.8e-4, 12e-4)
distribution_av.setDescription([r"$a_v$"])
#
distributionList = [
    distribution_alpha,
    distribution_beta,
    distribution_epsilon_e,
    distribution_epsilon_c,
    distribution_dp,
    distribution_gamma_p0,
    distribution_av,
]

distribution = ot.ComposedDistribution(distributionList)
# Workaround for https://github.com/openturns/openturns/issues/2255
labels = [marginal.getDescription()[0] for marginal in distributionList]

In [4]:
# Special focus is given to the hot leg
dataset_HL = ot.Sample.ImportFromCSVFile('../../data/SG_MC_HL.csv')

In [5]:
inputSample = dataset_HL[:,0:7]

In [6]:
outputSample = dataset_HL[:,7:]

In [7]:
inputSample

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=969 dimension=7 description=[data_0,data_1,data_2,...,data_4,data_5,data_6] data=[[105.516,0.0225637,0.427862,...,6.74309e-06,4.08487e-09,0.00061025],[105.472,0.0239598,0.257909,...,4.75128e-06,4.59559e-09,0.000831786],[95.0086,0.0234949,0.376177,...,3.62486e-06,3.23961e-09,0.000522635],...,[98.2262,0.0226106,0.27553,...,1.65602e-06,4.5673e-09,0.000574537],[99.8236,0.0239513,0.230916,...,3.95457e-06,1.33652e-09,0.000543576],[97.4609,0.0225135,0.365324,...,8.15759e-06,1.64233e-09,0.000444797]]

The following function creates a sparse polynomial chaos with a given total degree.



In [8]:
def ComputeSparseLeastSquaresChaos(
    inputTrain, outputTrain, multivariateBasis, totalDegree, distribution
):
    """
      Create a sparse polynomial chaos based on least squares.

      * Uses the enumerate rule in basis.
      * Uses the LeastSquaresStrategy to compute the coefficients based on
        least squares.
      * Uses LeastSquaresMetaModelSelectionFactory to use the LARS selection method.
      * Uses FixedStrategy in order to keep all the coefficients that the
        LARS method selected.

    Source : https://openturns.github.io/openturns/latest/auto_meta_modeling/polynomial_chaos_metamodel/plot_chaos_cv.html
    with a bug fix: replace getStrataCumulatedCardinal with getBasisSizeFromTotalDegree

      Parameters
      ----------
      inputTrain : Sample
          The input design of experiments.
      outputTrain : Sample
          The output design of experiments.
      multivariateBasis : Basis
          The multivariate chaos basis.
      totalDegree : int
          The total degree of the chaos polynomial.
      distribution : Distribution.
          The distribution of the input variable.

      Returns
      -------
      result : PolynomialChaosResult
          The estimated polynomial chaos.
    """
    selectionAlgorithm = ot.LeastSquaresMetaModelSelectionFactory()
    projectionStrategy = ot.LeastSquaresStrategy(
        inputTrain, outputTrain, selectionAlgorithm
    )
    enumerateFunction = multivariateBasis.getEnumerateFunction()
    basisSize = enumerateFunction.getBasisSizeFromTotalDegree(totalDegree)  # OK
    adaptiveStrategy = ot.FixedStrategy(multivariateBasis, basisSize)
    chaosalgo = ot.FunctionalChaosAlgorithm(
        inputTrain, outputTrain, distribution, adaptiveStrategy, projectionStrategy
    )
    chaosalgo.run()
    result = chaosalgo.getResult()
    return result

The following function computes the sparsity rate of the polynomial chaos.
To do this, we compute the number of coefficients in the decomposition assuming a linear enumeration rule and a fixed truncation.
The sparsity rate is the complement of the ratio between the number of coefficients
selected from LARS and the total number of coefficients in the full polynomial basis.



In [9]:
def computeSparsityRate(multivariateBasis, totalDegree, chaosResult):
    """Compute the sparsity rate, assuming a FixedStrategy."""
    # Get P, the maximum possible number of coefficients
    enumfunc = multivariateBasis.getEnumerateFunction()
    P = enumfunc.getStrataCumulatedCardinal(totalDegree)
    # Get number of coefficients in the selection
    indices = chaosResult.getIndices()
    nbcoeffs = indices.getSize()
    # Compute rate
    sparsityRate = 1.0 - nbcoeffs / P
    return sparsityRate

The following functions compute and plot the Q2 predictivity coefficients within the validation plot.



In [10]:
def computeQ2Chaos(chaosResult, inputTest, outputTest, outputMarginalIndex = 0):
    """Compute the Q2 of a chaos."""
    outputDimension = outputTest.getDimension()
    if outputMarginalIndex > outputDimension:
        raise ValueError("Output marginal index ", outputMarginalIndex, " is greater than the output dimension ", 
                         outputDimension)
    metamodel = chaosResult.getMetaModel()
    val = ot.MetaModelValidation(inputTest, outputTest, metamodel)
    Q2 = val.computePredictivityFactor()[outputMarginalIndex]
    Q2 = max(Q2, 0.0)  # We are not lucky every day.
    return Q2

In [11]:
def printChaosStats(multivariateBasis, chaosResult, inputTest, outputTest, totalDegree, outputMarginalIndex = 0):
    """Print statistics of a chaos."""
    outputDimension = outputTest.getDimension()
    if outputMarginalIndex > outputDimension:
        raise ValueError("Output marginal index ", outputMarginalIndex, " is greater than the output dimension ", 
                         outputDimension)
    sparsityRate = computeSparsityRate(multivariateBasis, totalDegree, chaosResult)
    Q2 = computeQ2Chaos(chaosResult, inputTest, outputTest, outputMarginalIndex)
    metamodel = chaosResult.getMetaModel()
    val = ot.MetaModelValidation(inputTest, outputTest, metamodel)
    graph = val.drawValidation().getGraph(0, outputMarginalIndex)
    legend1 = "O=%d, D=%d, Q2=%.2f%%" % (outputMarginalIndex, totalDegree, 100 * Q2)
    graph.setLegends(["", legend1])
    graph.setLegendPosition("topleft")
    print(
        "Degree=%d, Q2=%.2f%%, Sparsity=%.2f%%"
        % (totalDegree, 100 * Q2, 100 * sparsityRate)
    )
    return graph

In [12]:
multivariateBasis = ot.OrthogonalProductPolynomialFactory(
    distributionList
)

In [13]:
def compute_Q2_score_by_splitting(
    X, Y, basis, totalDegree, distribution, split_fraction=0.75
):
    """
    Compute Q2 score by splitting into train/test sets.

    Parameters
    ----------
    X : Sample(size, input_dimension)
        The X dataset.
    Y : Sample(size, output_dimension)
        The Y dataset.

    Returns
    -------
    Q2_score : float
        The Q2 score.
    """

    training_sample_size = X.getSize()
    X_train = ot.Sample(X)
    Y_train = ot.Sample(Y)
    split_index = int(split_fraction * training_sample_size)
    X_test = X_train.split(split_index)
    Y_test = Y_train.split(split_index)
    chaosResult = ComputeSparseLeastSquaresChaos(
        X_train, Y_train, basis, totalDegree, distribution
    )
    metamodel = chaosResult.getMetaModel()
    val = ot.MetaModelValidation(X_test, Y_test, metamodel)
    Q2_score = val.computePredictivityFactor()
    print('Here')
    Q2_local_mean = Q2_score.norm1() / Q2_score.getDimension()
    return Q2_local_mean

In [14]:
split_fraction = 0.75
training_sample_size = inputSample.getSize()
outputMarginalIndex = 0
fig = pl.figure(figsize=(25, 4))
for totalDegree in range(1, maxDegree + 1):
    print("totalDegree = ", totalDegree)
    print("  Train")
    X_train = ot.Sample(inputSample)
    Y_train = ot.Sample(outputSample)
    split_index = int(split_fraction * training_sample_size)
    X_test = X_train.split(split_index)
    Y_test = Y_train.split(split_index)
    chaosResult = ComputeSparseLeastSquaresChaos(
        X_train, Y_train, multivariateBasis, totalDegree, distribution
    )
    print("  Test")
    graph = printChaosStats(
        multivariateBasis, chaosResult, X_test, Y_test, totalDegree, outputMarginalIndex
    )
    ax = fig.add_subplot(1, maxDegree, totalDegree)
    _ = ot.viewer.View(graph, figure=fig, axes=[ax])
    pl.suptitle("Metamodel validation")

totalDegree =  1
  Train
  Test


WRN - set the component 26 of contributor 1=0 to zero as it is too small
WRN - set the component 53 of contributor 2=0 to zero as it is too small
WRN - set the component 54 of contributor 2=0 to zero as it is too small
WRN - set the component 55 of contributor 2=0 to zero as it is too small
WRN - set the component 56 of contributor 2=0 to zero as it is too small
WRN - set the component 57 of contributor 2=0 to zero as it is too small
WRN - set the component 58 of contributor 2=0 to zero as it is too small
WRN - set the component 59 of contributor 2=0 to zero as it is too small
WRN - set the component 60 of contributor 2=0 to zero as it is too small
WRN - set the component 61 of contributor 2=0 to zero as it is too small
WRN - set the component 62 of contributor 2=0 to zero as it is too small
WRN - set the component 63 of contributor 2=0 to zero as it is too small
WRN - set the component 64 of contributor 2=0 to zero as it is too small
WRN - set the component 65 of contributor 2=0 to ze

Degree=1, Q2=52.10%, Sparsity=12.50%
totalDegree =  2
  Train


In [ ]:
# TODO : loop over the output marginal indices ?
# TODO : grid of graphs with outputDimension rows ?

In [ ]:
# Q2_score = compute_Q2_score_by_splitting(inputSample, outputSample_BC, multivariateBasis, totalDegree)


We see that when the degree of the polynomial increases, the Q2 coefficient decreases.
We also see that the sparsity rate increases: while the basis size grows rapidly with the degree, the algorithm selects a smaller fraction of this basis.
This shows that the algorithm performs its task of selecting relevant coefficients.
However, this selection does not seem to be sufficient to mitigate the large number of coefficients.

Of course, this example is designed to make a predictivity decrease gradually.
We are going to see that this situation is actually easy to reproduce.



## Distribution of the predictivity coefficient



Let us repeat the following experiment to see the variability of the Q2 coefficient.



In [14]:
def computeSampleQ2(inputSample, outputSample, numberAttempts, maxDegree, split_fraction = 0.75):
    """
    For a given sample size N, for degree from 1 to maxDegree,
    repeat the following experiment numberAttempts times:
    create a sparse least squares chaos and compute the Q2
    using n_valid points.
    """
    sampleSize = inputSample.getSize()
    mixingDistribution = ot.KPermutationsDistribution(sampleSize, sampleSize)
    Q2sample = ot.Sample(numberAttempts, maxDegree)
    for totalDegree in range(maxDegree, maxDegree + 1):
        print("Degree = %d" % (totalDegree))
        for i in range(numberAttempts):
            # Randomize the sample
            X_train = ot.Sample(inputSample)
            Y_train = ot.Sample(outputSample)
            newIndices = mixingDistribution.getRealization()
            X_train = X_train[newIndices]
            Y_train = Y_train[newIndices]
            # Split
            split_index = int(split_fraction * sampleSize)
            X_test = X_train.split(split_index)
            Y_test = Y_train.split(split_index)
            # Train
            chaosResult = ComputeSparseLeastSquaresChaos(
                X_train, Y_train, multivariateBasis, totalDegree, distribution
            )
            # Test
            Q2sample[i, totalDegree - 1] = compute_Q2_score_by_splitting(
                X_test, Y_test, multivariateBasis, totalDegree, distribution, split_fraction=0.75
            )
    return Q2sample

In [ ]:
split_fraction = 0.75
training_sample_size = inputSample.getSize()
outputMarginalIndex = 0
fig = pl.figure(figsize=(25, 4))
for totalDegree in range(4, 5):
    print("totalDegree = ", totalDegree)
    print("  Train")
    X_train = ot.Sample(inputSample)
    Y_train = ot.Sample(outputSample)
    split_index = int(split_fraction * training_sample_size)
    X_test = X_train.split(split_index)
    Y_test = Y_train.split(split_index)
    chaosResult = ComputeSparseLeastSquaresChaos(
        X_train, Y_train, multivariateBasis, totalDegree, distribution
    )
    print("  Test")
    graph = printChaosStats(
        multivariateBasis, chaosResult, X_test, Y_test, totalDegree, outputMarginalIndex
    )
    ax = fig.add_subplot(1, maxDegree, totalDegree)
    _ = ot.viewer.View(graph, figure=fig, axes=[ax])
    pl.suptitle("Metamodel validation")

The following function uses a boxplot to see the distribution of the Q2 coefficients.



In [ ]:
numberAttempts = 15  # Number of repetitions

Q2sample = computeSampleQ2(inputSample, outputSample, numberAttempts, maxDegree)

In [ ]:
numberAttempts = 15  # Number of repetitions

Q2sample_1 = computeSampleQ2(inputSample, outputSample, numberAttempts, 4)

In [ ]:
def plotQ2Boxplots(Q2sample, N):
    data = np.array(Q2sample)
    pl.figure()
    pl.boxplot(data)
    pl.title(r"Hot branch at $z_{max}$, time-average of $Q^2$, PCE training sample size = %d" % N,fontsize=7)
    pl.xlabel(r"$p$")
    pl.ylabel(r"$m(Q^2)\;(\%)$")
    pl.ylim(0.0,1.0)
    return

Each experiment is repeated several times.



In [ ]:
Q2sample_1

We see that when the size of the design of experiments is as small as 20, it is more appropriate to use a very low degree polynomial. Here 1 performs best and 4 is risky.

In [ ]:
N = int(0.75*969)  # size of the train design
data_BF, data_BC = np.array(Q2sample), np.array(Q2sample_1)

fig, ax = pl.subplots(1,2,figsize=(10,5), layout='tight')
ax[0].boxplot(data_BC)
ax[0].set_title(r"Hot branch at $z_{max}$")
ax[0].set_ylim(0.0,1.0)
ax[0].set_xlabel(r"$p$")
ax[0].set_ylabel(r"$m(Q^2)\;(\%)$")

ax[1].boxplot(data_BF)
ax[1].set_title(r"Cold branch at $z_{max}$")
ax[1].set_ylim(0.0,1.0)
ax[1].set_xlabel(r"$p$")
ax[1].set_ylabel(r"$m(Q^2)\;(\%)$")

fig.suptitle(r"TPD-PCE time-average of $Q^2$ as a function of degree $p$")

fig.savefig('TPD_pce_degree_sensitivit.png',format='png')

With a 30-point design set, a polynomial degree of 2 is usually advisable.



In [ ]:
N = 50  # size of the train design
Q2sample = computeSampleQ2(N, n_valid, numberAttempts, maxDegree)
plotQ2Boxplots(Q2sample, N)

pl.show()

When the sample size increases, the Q2 computation becomes less sensitive to the polynomial degree.



## Conclusion

We observe that on the cantilever beam example, to use a polynomial total
degree equal to 4, we need a sample size at least equal to 50 to get a
satisfactory and reproducible Q2.
When the degree is equal to 4, if the sample is small, then depending on the
particular sample, the predictivity coefficient can be very low (i.e. less than 0.5).
With a sample size as small as 20, a polynomial degree of 1 is safer.
However the limited sample size may have an impact on other statistics that
could be derived from a metamodel calculated on such a small training sample.

## References

* "Metamodel-Based Sensitivity Analysis: Polynomial Chaos Expansions and Gaussian Processes", Loïc Le Gratiet, Stefano Marelli, Bruno Sudret,
  Handbook of Uncertainty Quantification, 2017, Springer International Publishing.


